In [241]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

y = train['Survived'].copy()
data = pd.concat([train.drop(['Survived'], axis=1), test], axis=0,ignore_index=True)

In [217]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 11 columns):
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Name           1309 non-null object
Sex            1309 non-null object
Age            1046 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1308 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 112.6+ KB


In [242]:
data['Embarked'].fillna('S', inplace=True)

In [325]:
mod = data['Fare'].median()
data['Fare'].fillna(mod, inplace=True)
c = pd.qcut(data['Fare'], 4)
data['fare_class'] = c

In [244]:
data['name_len'] = data['Name'].apply(lambda x: len(x))

In [245]:
data['title'] = data['Name'].apply( lambda x: str(x).split(',')[1].split()[0])
data['title'] = data['title'].apply(lambda x: x if x in ['Mr.', 'Mrs.', 'Miss.', 'Master.'] else ('live' if x in ['the','Mlle','Lady','Ms.','Sir.','Mme.'] else 'rare'))

In [246]:
data['Ticket'] = data['Ticket'].apply( lambda x: x.replace('.', ''))
data['Ticket'] = data['Ticket'].apply( lambda x: x.replace('/', ''))
data['tcat'] = data['Ticket'].apply( lambda x: 'NUM' if x.split()[0].isdigit() else x.split()[0])

In [541]:
data['tcat'] = data['tcat'].apply( lambda x: 'X' if x in ['SCA3','A','LP','PPP','SC','AQ4','SCAH','STONOQ','SCParis','AQ3','A4', 'SP', 'SOTONO2', 'SOPP', 'SOP', 'SCOW', 'Fa', 'SCA4', 'FC', 'AS', 'CASOTON'] else x)

In [247]:
a = data.groupby(['Pclass','title','Sex'])
b = a['Age'].transform('mean')
b.isna().sum()

1

In [248]:
b.fillna(b.mean(), inplace=True)
b.isna().sum()

0

In [249]:
b = data['Age'].fillna(b)
data['newage'] =  b.copy()
data['newage'].isna().sum()

0

In [400]:
data['age_class'] = pd.qcut(data['newage'],5)

In [250]:
data['Fam'] = data['SibSp'] + data['Parch']
data['famcat'] = data['Fam'].apply(lambda x: 'none' if x==0 else ( 'small' if (x==1) or (x==2) or (x==3) else 'large'))

In [251]:
data['cabincat'] = data['Cabin'].apply(lambda x: str(x)[0])

In [488]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 20 columns):
PassengerId    1309 non-null int64
Pclass         1309 non-null object
Name           1309 non-null object
Sex            1309 non-null object
Age            1046 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1309 non-null float64
Cabin          295 non-null object
Embarked       1309 non-null object
name_len       1309 non-null int64
title          1309 non-null object
tcat           1309 non-null object
newage         1309 non-null float64
Fam            1309 non-null int64
famcat         1309 non-null object
cabincat       1309 non-null object
fare_class     1309 non-null category
age_class      1309 non-null category
dtypes: category(2), float64(3), int64(5), object(10)
memory usage: 186.9+ KB


In [556]:
labels = ['Pclass', 'Sex', 'Embarked', 'Fare', 'title', 'tcat', 'newage', 'famcat', 'cabincat', 'SibSp', 'Parch'] 

In [256]:
data['Pclass'] = data['Pclass'].astype('str')

In [557]:
encode = list(data[labels].select_dtypes(include=["object", "category"]))
num = list(data[labels].select_dtypes(exclude=["object", "category"]))
encoded = pd.get_dummies(data[encode])
T = pd.concat([encoded, data[num]], axis = 1)

TRAIN = T[:len(train)]
TEST = T[len(train):]


# Models

In [27]:
from sklearn.model_selection import cross_val_score as CVS

### Feature Selection
We are dealing with a lot of features lets reduc them to reasonable ones first

In [307]:
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import RandomizedSearchCV as RSCV
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import StratifiedKFold

In [558]:
rfc = RFC(random_state=42, n_estimators=50,max_features='sqrt',  min_samples_split=6,min_samples_leaf=2, max_depth=20)

rfc.fit(TRAIN, y)

RandomForestClassifier(max_depth=20, max_features='sqrt', min_samples_leaf=2,
                       min_samples_split=6, n_estimators=50, random_state=42)

In [559]:
intro = pd.DataFrame({'score' : rfc.feature_importances_, 'features' : TRAIN.columns})
intro.sort_values(by='score')

,score,features
41,0.000000,cabincat_T
21,0.000000,tcat_PP
29,0.000058,tcat_WEP
40,0.000129,cabincat_G
23,0.000496,tcat_SOC
17,0.000529,tcat_FCC
24,0.000556,tcat_SOTONOQ
22,0.000582,tcat_SCPARIS
12,0.000608,title_live
39,0.000845,cabincat_F


There are lot of columns with next to nothing importance. We need to do feature selection to proceed

In [560]:
model = SelectFromModel(rfc, prefit=True)
TRAIN_R = model.transform(TRAIN)
print(TRAIN_R.shape)

TEST_R = model.transform(TEST)

(891, 12)


### Trying different models

In [300]:
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.linear_model import LogisticRegression as LR
from sklearn import svm

In [337]:
gbc = GBC()
knn = KNN(n_neighbors=5)
lr = LR(solver='lbfgs', max_iter=1500)
svc = svm.SVC(gamma=0.1, C=1)

In [338]:
models = [gbc, knn, lr, svc, rfc]

In [564]:
for model in models:
    
    val = CVS(model, TRAIN_R, y, cv = 5, scoring='accuracy')
    print('Model : ' + str(model.__class__ )+' Score : '+ str(np.mean(val)))
# Model : <class 'sklearn.ensemble._gb.GradientBoostingClassifier'> Score : 0.8327663046889713
# Model : <class 'sklearn.neighbors._classification.KNeighborsClassifier'> Score : 0.710488983742389
# Model : <class 'sklearn.linear_model._logistic.LogisticRegression'> Score : 0.822666499278137
# Model : <class 'sklearn.svm._classes.SVC'> Score : 0.7228359801644592
# Model : <class 'sklearn.ensemble._forest.RandomForestClassifier'> Score : 0.8338961772644529

Model : <class 'sklearn.ensemble._gb.GradientBoostingClassifier'> Score : 0.8327663046889713
Model : <class 'sklearn.neighbors._classification.KNeighborsClassifier'> Score : 0.710488983742389
Model : <class 'sklearn.linear_model._logistic.LogisticRegression'> Score : 0.822666499278137
Model : <class 'sklearn.svm._classes.SVC'> Score : 0.7228359801644592
Model : <class 'sklearn.ensemble._forest.RandomForestClassifier'> Score : 0.8338961772644529


AS expected Random Forest Model is turning out to be the best yet. But we need to tune hyperparameters.

In [567]:

params = {
    'max_depth' : [4, 6, 8, 20, 50],
    'n_estimators': [10, 30, 50, 60 ,],
    'max_features': ['sqrt', 'auto', 'log2'],
    'min_samples_split': [2, 3, 10],
    'min_samples_leaf': [1, 3, 10],
    'bootstrap': [True, False],
    }
rfc2 = RFC()
cross_validation = StratifiedKFold(n_splits=5)

search = RSCV(rfc2, scoring='accuracy', param_distributions = params,cv=cross_validation)

search.fit(TRAIN_R, y)

print('Best score: {}'.format(search.best_score_))

print('Best parameters: {}'.format(search.best_params_))

# Best score: 0.8316301550436256
# Best parameters: {'n_estimators': 30, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 4, 'bootstrap': True}

Best score: 0.8316301550436256
Best parameters: {'n_estimators': 30, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 4, 'bootstrap': True}


In [568]:
rfc2 = RFC(n_estimators= 30, min_samples_split = 2, min_samples_leaf = 1, max_features = 'auto', max_depth = 8, bootstrap = False)
rfc2.fit(TRAIN, y)

RandomForestClassifier(bootstrap=False, max_depth=8, n_estimators=30)

In [569]:

yp = rfc2.predict(TEST)
df = pd.DataFrame(yp, columns=['Survived'])
df1 = pd.concat( [test[['PassengerId']], df], axis=1)
df1.to_csv('new_titanic', index=False)